In [79]:
# Librerias
import random

In [80]:
# -------------------------- Calcular puntaje -----------------------
# Función para eliminar columnas de gaps
def eliminar_columnas_con_gaps(alineamiento):
    columnas_para_eliminar = []
    longitud_maxima = max(len(secuencia) for secuencia in alineamiento) # Num. maximo de caracteres en todas las secuencias.

    # Identifica las columnas que solo contienen gaps
    for i in range(longitud_maxima):  # Hasta la longitud máxima
        # Comprueba si todas las secuencias tienen un guion en la posición i
        columna_gap = all(
            i < len(alineamiento[j]) and alineamiento[j][i] == '-' for j in range(len(alineamiento))
        )
        if not columna_gap:
            columnas_para_eliminar.append(i)

    # Crear un nuevo alineamiento sin esas columnas
    nuevo_alineamiento = []
    for secuencia in alineamiento:
        nueva_secuencia = "".join(
            [secuencia[i] for i in columnas_para_eliminar if i < len(secuencia)]
        )
        nuevo_alineamiento.append(nueva_secuencia)

    return nuevo_alineamiento

# Función para calcular el puntaje de un alineamiento CORREGIR
def CalcularPuntaje(alineamiento, contador_secuencia):
    alineamiento_sin_gaps = eliminar_columnas_con_gaps(alineamiento)  # Elimina las columnas de gaps
    puntaje = 0
    longitud_sin_gaps = min(len(secuencia) for secuencia in alineamiento_sin_gaps)
    for i in range(longitud_sin_gaps):  # Solo esta puntuando hasta el minimo para evitar errores CORREGIR
        coincidencias = sum(
            alineamiento_sin_gaps[j][i] == alineamiento_sin_gaps[0][i] for j in range(contador_secuencia)
        )
        puntaje += coincidencias / contador_secuencia
    return puntaje
#--------------------------------------------------------------------

# ---------------------- Generar poblacion --------------------------
# Función para insertar un carácter en una posición específica
def insertar_caracter(cadena, caracter, posicion):
    return cadena[:posicion] + caracter + cadena[posicion:]

# Función para generar alineamientos aleatorios
def generar_random_alineamiento(secuencias):
    alineamiento = [secuencia for secuencia in secuencias]  # Clonar las secuencias originales
    for i in range(len(secuencias[0])):
        for j in range(len(secuencias)):
            if random.random() < 0.5:  # Decidir si se inserta un gap
                # Insertar un guion en la posición j en el índice actual (i)
                posicion_insercion = random.randint(0, i)  # Elegir una posición para insertar el guion
                alineamiento[j] = insertar_caracter(alineamiento[j], '-', posicion_insercion)
            # Siempre añadir el carácter actual de la secuencia
            if len(secuencias[j]) == i:
                alineamiento[j] += secuencias[j][i]
    return alineamiento
# --------------------------------------------------------------------

# --------------------------------- Cruza y mutacion de hijos ------------------------------------
# Función para ajustar el punto de corte (Para zigzaguear el guion)
def ajustar_punto_corte(secuencia, punto_corte):
    while punto_corte > 0 and secuencia[punto_corte - 1] == '-':
        punto_corte += 1  # Mover el punto de corte a la derecha
        if punto_corte >= len(secuencia):
            break
    return punto_corte

# Función para cruzar los alineamientos
def Cruza(padre1, padre2):
    longitud = len(padre1[0])

    # Elegir dos puntos de corte
    punto_corte1 = random.randint(0, longitud - 1)  # Primer punto de corte
    punto_corte2 = random.randint(punto_corte1 + 1, longitud)  # Segundo punto de corte

    # Ajustar los puntos de corte
    punto_corte1 = ajustar_punto_corte(padre1[0], punto_corte1)
    punto_corte2 = ajustar_punto_corte(padre1[0], punto_corte2)
    hijo1 = []
    hijo2 = []

    for j in range(len(padre1)):
        # Crear partes para el hijo 1
        izquierda_hijo1 = padre1[j][:punto_corte1]  # Parte izquierda de padre1
        medio_hijo1 = padre2[j][punto_corte1:punto_corte2]  # Parte media de padre2
        final_hijo1 = padre1[j][punto_corte2:]  # Parte final de padre1
        hijo1.append(izquierda_hijo1 + medio_hijo1 + final_hijo1)

        # Crear partes para el hijo 2
        izquierda_hijo2 = padre2[j][:punto_corte1]  # Parte izquierda de padre2
        medio_hijo2 = padre1[j][punto_corte1:punto_corte2]  # Parte media de padre1
        final_hijo2 = padre2[j][punto_corte2:]  # Parte final de padre2
        hijo2.append(izquierda_hijo2 + medio_hijo2 + final_hijo2)
    # print('Parte izquierda hijo1: ',izquierda_hijo1,
    #           "\nParte media hijo1:",medio_hijo1,
    #           "\nParte final hijo1:",final_hijo1)
    return hijo1, hijo2

# Función para aplicar mutación a un alineamiento
# Es literalmente la misma función que la de generar el alineamiento random, con la diferencia de que recorre la
# longitud maxima y recibe un valor de mutacion lo que se puede modificar desde fuera.
def mutacion(alineamiento, valor_mutacion):
    mutacion_alineamiento = ['' for _ in range(len(alineamiento))]  # Alineamiento vacío
    longitud_maxima = max(len(secuencia) for secuencia in alineamiento) # Longitud maxima
    for i in range(longitud_maxima):  
        for j in range(len(alineamiento)):                
            if i < len(alineamiento[j]) and random.random() < valor_mutacion:
                posicion_insercion = random.randint(0, i)  # Posición aleatoria
                mutacion_alineamiento[j] = insertar_caracter(mutacion_alineamiento[j], '-', posicion_insercion)
                mutacion_alineamiento[j] += alineamiento[j][i]
    return mutacion_alineamiento
# ----------------------------------------------------------------------------------------------

In [81]:
# Parámetros de entrada
secuencias = ["MURCIELAGO--", "ENUMERADO---","ESTRUENDO---","MUERDAGO----"]
tamaño_poblacion = 20
valor_mutacion = 0.3
generaciones = 1

# Creacion de la población
poblacion = [generar_random_alineamiento(secuencias) for _ in range(tamaño_poblacion)]

# Ciclo principal
for generation in range(generaciones):
    # Puntaje de aptitud de cada alineamiento
    coincidencia_puntajes = [CalcularPuntaje(alineamiento, len(secuencias)) for alineamiento in poblacion]
    
    # Selección de padres
    padres = random.choices(poblacion, weights=coincidencia_puntajes, k=2)
    
    # Cruzamiento
    hijo1, hijo2 = Cruza(padres[0], padres[1])
    
    # Mutación
    hijo1 = mutacion(hijo1, valor_mutacion)
    hijo2 = mutacion(hijo2, valor_mutacion)
    
    # Reemplazar los alineamientos
    poblacion[coincidencia_puntajes.index(min(coincidencia_puntajes))] = hijo1
    poblacion[coincidencia_puntajes.index(min(coincidencia_puntajes))] = hijo2

# Seleccionar el mejor alineamiento de la última generación
mejor_alineamiento = max(poblacion, key=lambda x: CalcularPuntaje(x, len(secuencias)))
print("Mejor alineamiento:")
for i, seq in enumerate(secuencias):
    print("Secuencia", i+1, ":", mejor_alineamiento[i])
print("Puntaje de aptitud:", CalcularPuntaje(mejor_alineamiento, len(secuencias)))


Parte izquierda hijo1:  -M----UER-D 
Parte media hijo1: AG 
Parte final hijo1: GO----
Mejor alineamiento:
Secuencia 1 : -----MU-RCIELAGO--
Secuencia 2 : --E--NU--MERADO---
Secuencia 3 : -------E--STRUENDO---
Secuencia 4 : -M--U-E-R-DAGO----
Puntaje de aptitud: 7.5
